In [ ]:
!pip install openfl


     |████████████████████████████████| 215kB 5.7MB/s 
     |████████████████████████████████| 153kB 14.1MB/s 
     |████████████████████████████████| 3.0MB 9.8MB/s 
     |████████████████████████████████| 645kB 47.2MB/s 
     |████████████████████████████████| 2.5MB 36.6MB/s 
     |████████████████████████████████| 3.2MB 35.0MB/s 
     |████████████████████████████████| 163kB 44.4MB/s 
     |████████████████████████████████| 204kB 49.1MB/s 
     |████████████████████████████████| 51kB 6.4MB/s 
  Created wheel for flatten-json: filename=flatten_json-0.1.13-cp37-none-any.whl size=7969 sha256=653b8aeedfcfef41628fa614049397d863130b9689dbba99d7380b479b622821
  Stored in directory: /root/.cache/pip/wheels/2f/55/29/7b552f0b137c7adb26493e5a6405c862c111352fe32ae91de8
Successfully built flatten-json
ERROR: tensorflow 2.4.1 has requirement grpcio~=1.32.0, but you'll have grpcio 1.30.0 which is incompatible.
  Found existing installation: grpcio 1.32.0
    Uninstalling grpcio-1.32.0:
      Succes

In [ ]:
!pip install torch torchvision mnist


In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
 
import torchvision
import torchvision.transforms as transforms
import openfl.native as fx
from openfl.federated import FederatedModel,FederatedDataSet


In [ ]:
fx.init('torch_cnn_mnist')

Creating Workspace Directories
Creating Workspace Templates

New workspace directory structure:
workspace
├── plan
│   ├── plan.yaml
│   ├── cols.yaml
│   ├── defaults
│   │   ├── tasks_fast_estimator.yaml
│   │   ├── tasks_keras.yaml
│   │   ├── collaborator.yaml
│   │   ├── network.yaml
│   │   ├── data_loader.yaml
│   │   ├── tasks_tensorflow.yaml
│   │   ├── defaults
│   │   ├── tasks_torch.yaml
│   │   ├── assigner.yaml
│   │   ├── aggregator.yaml
│   │   └── task_runner.yaml
│   └── data.yaml
├── requirements.txt
├── logs
├── .workspace
├── save
├── code
│   ├── ptmnist_inmemory.py
│   ├── __init__.py
│   ├── mnist_utils.py
│   └── pt_cnn.py
├── data
└── cert

7 directories, 20 files
Setting Up Certificate Authority...

1.  Create Root CA
1.1 Create Directories
1.2 Create Database
1.3 Create CA Request and Certificate
2.  Create Signing Certificate
2.1 Create Directories
2.2 Create Database
2.3 Create Signing Certificate CSR
2.4 Sign Signing Certificate CSR
3   Create Certificate

In [ ]:
def one_hot(labels, classes):
    return np.eye(classes)[labels]


In [ ]:
 
trf = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])


In [ ]:
train = torchvision.datasets.MNIST(root='./data', train=True,
                                 download=True, transform=trf)


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw


Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw
Processing...



Done!


/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:480: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [ ]:
t_img,t_label = train.train_data,np.array(train.train_labels)
 
#numpy.expand_dims() expands the shape of t_img array
#torch.from_numpy() forms a tensor from a numpy.ndarray
t_img = torch.from_numpy(np.expand_dims(t_img,  
axis=1)).float()
 
#Form one-hot encoding representation of labels by calling one_hot()  

t_label = one_hot(t_label,10) #10 is number of output classes


/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:58: UserWarning: train_data has been renamed data
  warnings.warn("train_data has been renamed data")
/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:48: UserWarning: train_labels has been renamed targets
  warnings.warn("train_labels has been renamed targets")


In [ ]:
valid = torchvision.datasets.MNIST(root='./data', train=False, download=True,transform=trf)

In [ ]:
v_img,v_label = valid.test_data, np.array(valid.test_labels)
v_img = torch.from_numpy(np.expand_dims(v_img, axis=1)).float()
v_label = one_hot(v_label,10)


/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:63: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:53: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


In [ ]:
feature_shape = t_img.shape[1]
classes = 10
 
#Prepare data for federation
data = FederatedDataSet(t_img,t_label,v_img,v_label,batch_size=32,
num_classes=classes)


In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, 3) #1st convolution
        self.pool = nn.MaxPool2d(2, 2) #pooling layer 
        self.conv2 = nn.Conv2d(16, 32, 3) #2nd convolution
	 #1st fully connected layer
        self.fc1 = nn.Linear(32 * 5 * 5, 32) 
        self.fc2 = nn.Linear(32, 84) #2nd fully connected layer
        self.fc3 = nn.Linear(84, 10) #3rd fully connected layer
 
    def forward(self, x):  #forward propagation
        x = self.pool(F.relu(self.conv1(x)))#pooling layer
        x = self.pool(F.relu(self.conv2(x))) #pooling layer
        x = x.view(x.size(0),-1)
        x = F.relu(self.fc1(x)) #activation for 1st FC layer
        x = F.relu(self.fc2(x)) #activation for 2nd FC layer
        x = self.fc3(x)
        return F.log_softmax(x, dim=1)


In [ ]:
opt = lambda x: optim.Adam(x, lr=1e-4)

In [ ]:
def cross_entropy(output, target):
    """Binary cross-entropy metric
    """
    return F.binary_cross_entropy_with_logits(input=output,target=target)


In [ ]:
fmodel = FederatedModel(build_model=Net,optimizer=opt,
loss_fn=cross_entropy,data_loader=data)


[14:00:16] WARNING  tried to remove tensor: __opt_state_needed not present in the tensor dict                                                        utils.py:86


In [ ]:
c_models = fmodel.setup(num_collaborators=2)
#Define which collaborators will take part in the experiment
collaborators = {'one':c_models[0],'Two':c_models[1]}


[14:01:08] WARNING  tried to remove tensor: __opt_state_needed not present in the tensor dict                                                        utils.py:86
           WARNING  tried to remove tensor: __opt_state_needed not present in the tensor dict                                                        utils.py:86


In [ ]:
#Original MNIST dataset
print(f'Original training data size: {len(t_img)}')
print(f'Original validation data size: {len(v_img)}\n')
 
#1st collaborator’s data
print(f'Collaborator 1\'s training data size: {len(c_models[0].data_loader.X_train)}')
print(f'Collaborator 1\'s validation data size: {len(c_models[0].data_loader.X_valid)}\n')
 
#2nd collaborator's data
print(f'Collaborator 2\'s training data size: {len(c_models[1].data_loader.X_train)}')
print(f'Collaborator 2\'s validation data size: {len(c_models[1].data_loader.X_valid)}\n')


Original training data size: 60000
Original validation data size: 10000

Collaborator 1's training data size: 30000
Collaborator 1's validation data size: 5000

Collaborator 2's training data size: 30000
Collaborator 2's validation data size: 5000



In [ ]:
import json
#fx.get_plan() command returns all the plan values that can be set
print(json.dumps(fx.get_plan(), indent=4, sort_keys=True))
 


{
    "aggregator.settings.best_state_path": "save/torch_cnn_mnist_best.pbuf",
    "aggregator.settings.db_store_rounds": 1,
    "aggregator.settings.init_state_path": "save/torch_cnn_mnist_init.pbuf",
    "aggregator.settings.last_state_path": "save/torch_cnn_mnist_last.pbuf",
    "aggregator.settings.rounds_to_train": 10,
    "aggregator.template": "openfl.component.Aggregator",
    "assigner.settings.task_groups": [
        {
            "name": "train_and_validate",
            "percentage": 1.0,
            "tasks": [
                "aggregated_model_validation",
                "train",
                "locally_tuned_model_validation"
            ]
        }
    ],
    "assigner.template": "openfl.component.RandomGroupedAssigner",
    "collaborator.settings.db_store_rounds": 1,
    "collaborator.settings.delta_updates": false,
    "collaborator.settings.opt_treatment": "RESET",
    "collaborator.template": "openfl.component.Collaborator",
    "data_loader.settings.batch_size": 2

In [ ]:
final_model = fx.run_experiment(collaborators,{'aggregator.settings.rounds_to_train':5})


[14:04:18] INFO     Updating aggregator.settings.rounds_to_train to 5...                                                                           native.py:106
           INFO     Loading DEFAULTS for section aggregator from file plan/defaults/aggregator.yaml.                                                  plan.py:95
           INFO     Loading DEFAULTS for section assigner from file plan/defaults/assigner.yaml.                                                      plan.py:95
           INFO     Loading DEFAULTS for section collaborator from file plan/defaults/collaborator.yaml.                                              plan.py:95
           INFO     Loading DEFAULTS for section data_loader from file plan/defaults/data_loader.yaml.                                                plan.py:95
           INFO     Loading DEFAULTS for section network from file plan/defaults/network.yaml.                                                        plan.py:95
           INFO     Loading DEFAUL

/usr/local/lib/python3.7/dist-packages/openfl/federated/task/runner_pt.py:98: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  data, target = pt.tensor(data).to(self.device), pt.tensor(


[14:04:20] INFO     Sending metric for task aggregated_model_validation, round number 0: acc        0.055                                    collaborator.py:366
           INFO     Collaborator one is sending task results for aggregated_model_validation, round 0                                          aggregator.py:430


/usr/local/lib/python3.7/dist-packages/openfl/federated/task/runner_pt.py:152: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  data, target = pt.tensor(data).to(self.device), pt.tensor(


[14:04:35] INFO     Sending metric for task train, round number 0: cross_entropy    0.12506341934204102                                      collaborator.py:366
           INFO     Collaborator one is sending task results for train, round 0                                                                aggregator.py:430
[14:04:36] INFO     Sending metric for task locally_tuned_model_validation, round number 0: acc     0.9488                                   collaborator.py:366
           INFO     Collaborator one is sending task results for locally_tuned_model_validation, round 0                                       aggregator.py:430
           INFO     All tasks completed on one for round 0...                                                                                collaborator.py:150
           INFO     Waiting for tasks...                                                                                                     collaborator.py:156
           INFO     Sending tasks 